# LeNet Computer Vision Model

> https://doi.org/10.1109/5.726791

## Import modules and set random seed

Seed is set for reproducible results

In [1]:
from datetime import datetime
from pathlib import Path
from typing import Any, List

import numpy
import tensorflow

from keras import layers, losses
from keras.callbacks import TensorBoard
from keras.datasets.mnist import load_data
from keras.models import Sequential
from numpy import ndarray


numpy.random.seed(seed=42)

2023-04-24 13:52:11.995845: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 13:52:11.997353: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-24 13:52:12.027638: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-24 13:52:12.028533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 13:52:12.613605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Download and preparte MNIST dataset

1. As the `LeNet` model expects images to be of size *32 x 32*, all images within the *MNIST* dataset need to be scaled from *28 x 28* to *32 x 32*
2. As *MNIST* images are in grayscale, we want to binarize them between 0 and 1 (white or black) by dividing their color value by 255
3. As *MNIST* images are in grayscale, they do not have the color channel value that is expected by *Keras* `Conv2d` module. In other words, the *MNIST* dataset tensor structure only contains [`batchSize`, `height`, `width`] Thus, we need to add in a fourth dimension to make our tenors look like [`batchSize`, `height`, `width`, `channel`] where `channel` == 1

In [2]:
imagePadding: List[List[int]] = [[0, 0], [2, 2], [2, 2]]

mnist: tuple[tuple[Any, Any]] = load_data()

xTrain: ndarray = mnist[0][0]
yTrain: ndarray = mnist[0][1]
xTest: ndarray = mnist[1][0]
yTest: ndarray = mnist[1][1]

xTrain = tensorflow.pad(tensor=xTrain, paddings=imagePadding) / 255
xTest = tensorflow.pad(tensor=xTest, paddings=imagePadding) / 255

xTrain = tensorflow.expand_dims(input=xTrain, axis=3)
xTest = tensorflow.expand_dims(input=xTest, axis=3)

2023-04-24 13:52:13.502982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 13:52:13.503206: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Build the model

### Architecture

[![https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)

> Image from https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg

In [3]:
lenet: Sequential = Sequential(name="LeNet")
lenet.add(layer=layers.Conv2D(filters=6, kernel_size=5, activation="tanh"))
lenet.add(layers.AveragePooling2D(pool_size=2))
lenet.add(layer=layers.Activation(activation="sigmoid"))
lenet.add(layers.Conv2D(16, 5, activation="tanh"))
lenet.add(layers.AveragePooling2D(2))
lenet.add(layers.Activation("sigmoid"))
lenet.add(layers.Conv2D(120, 5, activation="tanh"))
lenet.add(layers.Flatten())
lenet.add(layers.Dense(84, activation="tanh"))
lenet.add(layers.Dense(10, activation="softmax"))
lenet.build(input_shape=xTrain.shape)
lenet.compile(
    optimizer="adam",
    loss=losses.sparse_categorical_crossentropy,
    metrics=["accuracy"],
)
lenet.summary()

Model: "LeNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (60000, 28, 28, 6)        156       
                                                                 
 average_pooling2d (AverageP  (60000, 14, 14, 6)       0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (60000, 14, 14, 6)        0         
                                                                 
 conv2d_1 (Conv2D)           (60000, 10, 10, 16)       2416      
                                                                 
 average_pooling2d_1 (Averag  (60000, 5, 5, 16)        0         
 ePooling2D)                                                     
                                                                 
 activation_1 (Activation)   (60000, 5, 5, 16)         0     

## Train the model

In [4]:
logFolder: Path = Path("logs/lenet-" + datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_callback: TensorBoard = TensorBoard(
    log_dir=logFolder,
    histogram_freq=1,
    write_images=True,
)

lenet.fit(
    x=xTrain,
    y=yTrain,
    batch_size=64,
    epochs=10,
    callbacks=[tensorboard_callback],
    validation_split=0.15,
)

Epoch 1/10
797/797 [==============================] - 6s 7ms/step - loss: 1.7212 - accuracy: 0.3616 - val_loss: 0.4350 - val_accuracy: 0.8607
Epoch 2/10
797/797 [==============================] - 5s 6ms/step - loss: 0.3651 - accuracy: 0.8833 - val_loss: 0.3156 - val_accuracy: 0.8996
Epoch 3/10
797/797 [==============================] - 5s 6ms/step - loss: 0.2762 - accuracy: 0.9113 - val_loss: 0.2120 - val_accuracy: 0.9324
Epoch 4/10
797/797 [==============================] - 5s 6ms/step - loss: 0.2300 - accuracy: 0.9265 - val_loss: 0.1883 - val_accuracy: 0.9399
Epoch 5/10
797/797 [==============================] - 5s 6ms/step - loss: 0.2012 - accuracy: 0.9365 - val_loss: 0.1753 - val_accuracy: 0.9448
Epoch 6/10
797/797 [==============================] - 5s 6ms/step - loss: 0.1794 - accuracy: 0.9418 - val_loss: 0.1481 - val_accuracy: 0.9554
Epoch 7/10
797/797 [==============================] - 5s 6ms/step - loss: 0.1662 - accuracy: 0.9481 - val_loss: 0.1318 - val_accuracy: 0.9600
Epoch 

## Evaluate the model on the testing dataset

In [5]:
lenet.evaluate(
    x=xTest,
    y=yTest,
    batch_size=64,
    callbacks=[tensorboard_callback],
)

157/157 [==============================] - 0s 3ms/step - loss: 0.1190 - accuracy: 0.9649


[0.11903892457485199, 0.964900016784668]

## Save the model to disk

In [6]:
outputPath: Path = Path("models/lenet").resolve()
lenet.save(
    filepath=outputPath,
    overwrite=True,
    save_format="tf",
)

INFO:tensorflow:Assets written to: /home/nsynovic/documents/projects/personal/dl-examples/dl_examples/models/lenet/assets


INFO:tensorflow:Assets written to: /home/nsynovic/documents/projects/personal/dl-examples/dl_examples/models/lenet/assets
